In [3]:
import cv2
import numpy as np
import gradio as gr

def cartoonize_image_logic(img, intensity):
    """
    核心處理邏輯
    img: 輸入圖片 (BGR format from OpenCV)
    intensity: 滑桿數值 (0-100)，控制卡通化強弱
    """
    # 根據滑桿數值動態調整參數
    # 強度越高 -> 模糊次數越多 (filter_iterations)
    # 強度越高 -> 色塊越平滑 (sigmaColor)
    # 強度越高 -> 邊緣線條越粗曠 (blockSize)
    
    # 參數映射邏輯
    filter_iterations = int(intensity / 10) + 1  # 範圍 1~11
    sigmaColor = int(intensity * 2) + 10         # 範圍 10~210
    sigmaSpace = 7
    
    # blockSize 必須是奇數，且大於 1
    # 這裡將 intensity 映射到 5~25 之間的奇數
    base_block = int(intensity / 5)
    blockSize = base_block if base_block % 2 == 1 else base_block + 1
    if blockSize < 3: blockSize = 3
    
    C = 2 
    
    # 縮小圖片以加速處理並減少雜訊
    img_small = cv2.pyrDown(cv2.pyrDown(img))

    # 1. 雙邊模糊 (Bilateral Filter) - 產生油畫般的平滑色塊
    for _ in range(filter_iterations):
        img_small = cv2.bilateralFilter(img_small, d=9, sigmaColor=sigmaColor, sigmaSpace=sigmaSpace)

    # 放大回原尺寸
    img_cartoon = cv2.pyrUp(cv2.pyrUp(img_small))
    
    # 確保尺寸一致 (因為 pyrDown/Up 可能會有 1 pixel 的誤差)
    img_cartoon = cv2.resize(img_cartoon, (img.shape[1], img.shape[0]))

    # 2. 邊緣偵測 (Edge Detection)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_blur = cv2.medianBlur(img_gray, 7)
    
    img_edge = cv2.adaptiveThreshold(
        img_blur, 255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY,
        blockSize=blockSize,
        C=C
    )
    
    # 轉換邊緣圖為彩色格式以便合併
    img_edge_colored = cv2.cvtColor(img_edge, cv2.COLOR_GRAY2BGR)

    # 3. 合成 (Bitwise AND)
    cartoon = cv2.bitwise_and(img_cartoon, img_edge_colored)

    return cartoon

def process_frame(input_image, slider_value):
    """
    Gradio 介面呼叫的函數
    Gradio 傳入的圖片通常是 RGB (numpy array)
    """
    if input_image is None:
        return None
    
    # Gradio 讀入的是 RGB，OpenCV 處理通常用 BGR (雖然這裡沒存檔沒差，但習慣轉換一下)
    # 為了保持色彩正確，我們先轉 BGR 處理，最後回傳 RGB 給網頁顯示
    img_bgr = cv2.cvtColor(input_image, cv2.COLOR_RGB2BGR)
    
    # 執行卡通化
    output_bgr = cartoonize_image_logic(img_bgr, slider_value)
    
    # 轉回 RGB 供介面顯示
    output_rgb = cv2.cvtColor(output_bgr, cv2.COLOR_BGR2RGB)
    
    return output_rgb

# --- 建立 Gradio GUI 介面 ---

with gr.Blocks(title="影像卡通化轉換器") as demo:
    gr.Markdown("# 🎨 影像卡通化轉換器")
    
    with gr.Row():
        with gr.Column():
            # 左側：輸入區
            input_img = gr.Image(label="上傳圖片", type="numpy")
            # 滑桿：控制強度
            intensity_slider = gr.Slider(minimum=0, maximum=100, value=50, step=5, label="卡通化程度")
            submit_btn = gr.Button("開始轉換", variant="primary")
            
        with gr.Column():
            # 右側：輸出區
            output_img = gr.Image(label="卡通化結果")

    # 設定按鈕事件
    submit_btn.click(
        fn=process_frame, 
        inputs=[input_img, intensity_slider], 
        outputs=output_img
    )

# 啟動介面
demo.launch()

Running on local URL:  http://127.0.0.1:7883

To create a public link, set `share=True` in `launch()`.
